In [ ]:
# Main program for nse

from ib_insync import *

import logging
import pandas as pd
from os import listdir

from helper import get_connected, grp_opts, get_prec, catch
from nse_func import p_nses, p_nseopts, upd_nses, upd_opt, remqty_nse, target_nse, nse_tgt_update

from nse_variables import *

util.startLoop()  # should not be there in py file

keep_pickles = True   # keep already pickled symbols

ib = get_connected('nse', 'live') # close the connection and delete it final function

# Weekend update functions
# __________________________

def weekend_process(ib):
    '''Weekend process to generate underlyings (lot+margin) and optio pickles
    Arg: (ib) as connection
    Returns: 
        None
        But pickles to _lot_margin.pickle & (SYMBOL).pkl'''
    
    util.logToFile(fspath+'_errors.log')  # create log file
    with open(fspath+'_errors.log', 'w'): # clear the previous log
        pass
    
    # Step # 1: generate the underlyings
    dfu = p_nses(ib).dropna()

    # Step # 2: generate the option pickles
    [catch(lambda: p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) 
     for u, p, z, m in zip(dfu.contract, dfu.undPrice, dfu.lot, dfu.margin))]
    
    return None
    
def pickle_remaining_opts(ib, keep_pickles = True):
    '''Upon any failure, pickle remaining options upon failure
    Arg: 
        (ib) as connection
        <keep_pickles> as True / False for keeping existing pickles
    Returns:
       None'''
    
    util.logToFile(fspath+'_errors.log')  # create log file
    
    # get the underlying dataframe
    dfu = pd.read_pickle(fspath+'_lot_margin.pickle')
    
    if keep_pickles:
        fs = listdir(fspath)
        optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']
        dfr = dfu[~dfu.index.isin(optsList)] # remaining dfs (without existing pickles)
        [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m)
         for u, p, z, m in zip(dfr.contract, dfr.undPrice, dfr.lot, dfr.margin)]
    else:
        [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m)
         for u, p, z, m in zip(dfu.contract, dfu.undPrice, dfu.lot, dfu.margin)]
        
    return None

# Everyday update function
# ________________________

def everyday_process(ib):
    '''everyday process to update update underlyings, option prices and generate targets
    (takes about 40 mins)
    Arg: (ib) as connection
    Returns:
       None
       But pickles lot and margin updates to underlyings
    '''
    util.logToFile(fspath+'_errors.log')  # create log file
    
    # ...update underlyings
    dfu = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()
    upd_nses(ib, dfu)
    
    # ...update option prices
    from helper import upd_opt
    fs = listdir(fspath)
    
    # collect the options
    optsList = [f for f in fs if f[-3:] == 'pkl']
    dfopts = pd.concat([pd.read_pickle(fspath+f+'.pkl')
                     for f in optsList], axis=0, sort=True).reset_index(drop=True)

    # update the option prices
    df = upd_opt(ib, dfopts)
    
    # ...generate targets and pickle, with new remaining quantities
    dft = target_nse(ib)
    
    return None

# Dynamic update function
# _______________________

def dynamic_update(ib):
    '''dynamically update the target with price and remaining quantities
    Arg: (ib) as connection object
    Returns: dft as DataFrame and re-pickles it
       '''
    
    return nse_tgt_update(ib)

In [ ]:
%%time
# Run the weekend process
weekend_process(ib)

In [ ]:
%%time
# Run everyday_process
everyday_process(ib)

In [ ]:
%%time
# Run dynamic pricess
dft = dynamic_update(ib).reset_index()

In [ ]:
%%time
ib.openOrders()

In [ ]:
dft = pd.read_pickle(fspath+'_targetopts.pickle')

In [ ]:
df_trades = pd.concat([pd.DataFrame(trades), util.df(trades)], axis=1)
df_trades.columns=[['trade']+list(df_trades)[1:]]
df_trades.to_pickle(fspath+'_tradesDone.pickle')

In [ ]:
# One scrip check (TEST MODULE)

dfu = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

fs = listdir(fspath)
optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']
dfr = dfu[~dfu.index.isin(optsList)] # remaining dfs (without existing pickles)